### Voting Classifier
- 다른 알고리즘 적용
- 다수결 예측

In [2]:
import numpy as np
import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [8]:
cancer = load_breast_cancer()
cancer.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [9]:
# Data Loading
cancer = load_breast_cancer()
data_df = pd.DataFrame(
    cancer.data
    ,columns=cancer.feature_names
)
data_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [10]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [11]:
# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.2, random_state=156)

In [18]:
# 모델 객체 생성
lr_clf = LogisticRegression(max_iter=10000)
knn_clf = KNeighborsClassifier(n_neighbors=8)

# Voting 객체 생성
vo_clf = VotingClassifier(
      estimators=[('LR',lr_clf),('KN',knn_clf)]  # 사용할 모델 알고리즘 [(내부이름변수명 지정, 모델객체),()]
    , voting='soft'     # Voting 방식 지정(Hard, soft-디폴트)
)

In [15]:
pd.Series(y_train).value_counts()
# 균일하지 않지만, 불균형 데이터셋이라고는 할수 없다. 그러나 의료데이터이기 때문에 재현율(Recall) 중요. 그러나 책에서는 정확도만 다룸.

1    280
0    175
dtype: int64

In [19]:
# Voting 객체 학습
vo_clf.fit(X_train, y_train)

# 예측
pred=vo_clf.predict(X_test)

accu=accuracy_score(y_test, pred)
print('정확도: {0:.4f}'.format(accu))

정확도: 0.9474


In [20]:
# 모델별 정확도 확인
clf_obj=[lr_clf, knn_clf]

for clf in clf_obj:
    clf.fit(X_train, y_train)
    pred=clf.predict(X_test)
    class_name=clf.__class__.__name__
    accu=accuracy_score(y_test, pred)
    print('{0} 정확도:{1:.4f}'.format(class_name, accu))

LogisticRegression 정확도:0.9649
KNeighborsClassifier 정확도:0.9386


In [26]:
lr_clf.__class__

sklearn.linear_model._logistic.LogisticRegression

In [27]:
lr_clf.__class__.__name__

'LogisticRegression'